In [1]:
%run "../layeringOfServices.ipynb"

In [2]:
#import os
#from datetime import datetime
#from datetime import date
import pymysql
from sqlalchemy import create_engine
from decouple import config 
from dotenv import load_dotenv
#import pandas as pd
#from numpy import int16
#from enum import Enum

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option("max_columns",None)


In [3]:
load_dotenv()
# get the environment variables needed
USER= config('USRCaris')
PASSWORD= config('PASSCaris')
HOSTNAME= config('HOSTCaris')
DBNAME= config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")
query = f'''
    SELECT 
    gb.* 
    FROM caris_db.gardening_beneficiary gb 
    where  
    gb.beneficiary_type ='dreams'
'''

gbd = pd.read_sql_query(query,engine,parse_dates=True)
# get the test excel file from Query

# close the pool of connection
engine.dispose()

True

In [4]:
gbd.to_excel("data_gardens.xlsx",index=False,na_rep="NULL")

In [5]:
df = dreams_mastersheet

In [6]:
cohorte = df[
    (df.age_range != "25-29")&
    (df.age_range != "not_valid_age")&
    (df.timeOn_system=="required_Time_on")
]


jardinage = df[
    (df.age_range != "25-29")&
    (df.age_range != "not_valid_age")&
    (df.timeOn_system=="required_Time_on")&
    (df.is_gardening=="yes")
]

cohorte.code.count()
gbd.case_id.count()
jardinage.code.count()

17510

1460

319

In [7]:
lgb_old = pd.read_excel("liste_filles_dreams_eligibles_jardinage_24Mai2021.xlsx")

In [8]:
lgb_old.code.count()

9870

## HOw MUCH

In [9]:
gbd = gbd.loc[:,~gbd.columns.duplicated()]

In [10]:
how_much_isin_lgb_old = gbd[gbd.code_dreams.isin(lgb_old.code)]
how_much_isin_lgb_old.code_dreams.count()

221

In [11]:
how_much_isin_jardinage = gbd[gbd.code_dreams.isin(jardinage.code)]
how_much_isin_jardinage.code_dreams.count()

343

In [12]:
gbd.duplicated(subset=['code_dreams'])

0       False
1       False
2        True
3       False
4        True
        ...  
1455    False
1456     True
1457     True
1458    False
1459     True
Length: 1460, dtype: bool

# Radicial MOVE

In [13]:
garden_served = pd.read_excel("./Gardening_AGYW_served_From April 2020.xlsx",sheet_name="Filles_Dreams_From_April_2020")

In [14]:
garden_served.code_dreams.fillna("NULL",inplace=True)
#garden_served.code_dreams = garden_served.code_dreams.str.upper()
garden_served.code_dreams.count()

1460

In [15]:
garden_served["validation_code_dreams"]= garden_served.code_dreams.str.fullmatch("^[A-Z]{3}/DRMS/(\d{9})$")

In [16]:
garden_served["duplication_code_dreams"]=  garden_served.code_dreams.duplicated()

In [17]:
garden_served.to_excel("validation_garden.xlsx",index=False,na_rep="NULL")

## Applied ON GBD

In [18]:
gbd.code_dreams.fillna("NULL",inplace=True)
#gbd.code_dreams = gbd.code_dreams.str.upper()
gbd.code_dreams.count()

gbd["validation_code_dreams"]= gbd.code_dreams.str.fullmatch("^[A-Z]{3}/DRMS/(\d{9})$")
gbd["duplication_code_dreams"]=  gbd.code_dreams.duplicated()

gbd

1460

,case_id,closed,closed_by,date_closed,date_modified,domain,id,indexed_on,indices,opened_by,resource_uri,server_date_modified,server_date_opened,user_id,xform_ids,address,address_commune,address_department,address_zone,age,backend_id,beneficiary_type,caris_site,case_name,case_type,cycle_number,date_opened,external_id,gps,inserted_at,last_name,owner_id,owner_type,patient_code,start_date,telephone,age_bracket,cycle_1_end_date,cycle_1_start_date,cycle_2_start_date,end_date,fail_date_cycle_1,neighbor_code,visit_number,first_name,next_cycle_elligibility,hh_date,hm_household_size,hs_boys_10_to_14,hs_boys_15_to_17,hs_boys_18_to_24,hs_boys_1_to_4,hs_boys_25_plus,hs_boys_5_to_9,hs_boys_less_than_1_year,hs_girls_10_to_14,hs_girls_15_to_17,hs_girls_18_to_24,hs_girls_1_to_4,hs_girls_25_plus,hs_girls_5_to_9,hs_girls_less_than_1_year,garden_pre_questionnaire_date,hs_persons_over_18,hs_persons_under_18,garden_established_date,garden_post_questionnaire_cycle_1,cycle_2_end_date,fail_date_cycle_2,garden_post_questionnaire_cycle_2,gps_cycle_2,cycle_3_start_date,cycle_3_end_date,dat_nesans,hs_other_person_responsible,hs_responsible_person,timoun_nan_kay_la,total_beneficiaire,total_beneficiaire_under18,totalcaregiverfi1824,totalcaregiverfi25plus,totalcaregivergason1824,totalcaregivergason25plus,ki_laj_fi_a,eske_gen_lot_moun_nan_kay_la_ki_responsab_timoun_yo,laj_lot_ki_laj_fi_a,ki_lajlot_gason_an,ki_lajlot_gason_an2,dat_gradyasyon,ki_kote_gradyasyon_an_fet,localisation_gradyasyon,nan_ki_depatman_ak_ki_komin_gradyasyon,temwanyaj_benefisye_gradyasyon,cycle_4_start_date,start_date_Cycle_2,date_suivi_telefonik,eske_jaden_an_toujou_egziste,eske_ou_dako_reponn_kesyon_kovid,eske_ou_konn_kijan_pou_pwotejew,eske_ou_konprann_sa_ki_maladi_kovid_19_korona_a,eske_ou_refe_yon_lot_jaden,eske_ou_rete_lakay_ou_poutet_korona,eskew_te_reyalize_yon_bon_rekot,konesans_sentom_korona_viris,nan_ki_eta_jaden_an_ye,nan_ki_faz_jaden_an_ye_kounyea,cycle_4_end_date,poukisa_jaden_an_pa_egziste_anko,konbyen_mwa_enstalasyon_lot_jaden,konbyen_tan_an_jou_semenn_mwa,konbyen_semenn_enstalasyon_lot_jaden,cycle_2_fail_Date,code_dreams,code_safe_space,konbyen_jou_enstalasyon_lot_jaden,validation_code_dreams,duplication_code_dreams
0,0063ab09-151f-4dfc-b2a0-a0e278c13a41,0,None,NaT,2021-01-07 12:55:21,caris-test,None,NaT,{},None,None,2021-01-07 18:21:19,2020-09-22 19:28:11,6fd64b1860874ed0a528eb9176102f56,"[""16caaece-17eb-4b52-9a1b-e02ad13f6232"",""881fc...",Achin 2 Verrettes,Verrettes,artibonite,Achin 2,37,None,dreams,ZLS/VERR - Hopital Dumarsais Estime,Présita,Garden,1,2020-09-10T13:57:24.739,,19.0471896 -72.4767567 60.0 11.0,None,Prévilmat,6fd64b1860874ed0a528eb9176102f56,None,None,2020-09-08,47625830,25_plus,None,2020-11-12,None,None,None,None,2,None,None,2020-11-12,6,1,0,0,0,1,0,0,1,1,0,0,0,2,0,2020-09-29,None,None,None,None,None,None,None,None,None,None,None,laj_gason_reskonsab,non,oui_gen_timoun,6,5,0,0,0,1,None,non_pagen_lot_reskonsab,None,40,40,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NULL,None,None,False,False
1,00685e25-9238-4125-bf17-86a828ef7961,0,None,NaT,2021-03-15 13:20:59,caris-test,None,NaT,{},None,None,2021-03-22 12:31:10,2020-11-16 13:37:38,d47dc50151de47efa1f00fdaa4c7eb4d,"[""460b4243-38f5-478f-96c9-efe566c2dbb4"",""315e7...",,Grande-Saline,artibonite,danache,24,None,dreams,ZLS/STMC - SSPE de St Marc,milancia,Garden,1,2020-11-15T12:13:54,,19.2240177 -72.6209523 -11.41 9.0,None,dorval,d47dc50151de47efa1f00fdaa4c7eb4d,None,None,2020-11-15,42317795,18_24,None,2021-02-05,None,None,None,None,4,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,None,None,False,False
2,009d89b9-39a3-422a-9301-b77b98ec3c6a,0,None,